In [1]:
import numpy as np
import scipy
import common

import matplotlib.pyplot as plt
import seaborn as sns

from ConfigParser import SafeConfigParser

parser = SafeConfigParser()
parser.read("../config.ini")

width = int(parser.get("Sampling", "width"))
height = int(parser.get("Sampling", "height"))

amp_min = float(parser.get("Sampling", "amp_min"))
amp_max = float(parser.get("Sampling", "amp_max"))

rad_min = float(parser.get("Sampling", "rad_min"))
rad_max = float(parser.get("Sampling", "rad_max"))

prefix = parser.get("Misc", "prefix")
location = parser.get("Misc", "location")
output_folder = location + "/" + prefix 

x,y,r,a,L = np.loadtxt(output_folder + "/" + prefix + "_out_points_som.txt", unpack=True)

all_vals = np.vstack((x,y,r,a,L))

In [2]:
%matplotlib inline

In [3]:
def uniqueish_color():
    """There're better ways to generate unique colors, but this isn't awful."""
    return plt.cm.gist_ncar(np.random.random())

In [4]:
bin_amt = 350

In [5]:

def next_dim(dim):
    if dim == 0:
        return 1
    return 0


In [6]:
#def get_peaks(start, stop, dim, debug_plots = True):
def get_peaks(bounds, dim, debug_plots = False):
    start, stop = bounds[dim]
    other_start, other_stop = bounds[next_dim(dim)]
    
    if start == stop:
        return []
    lvals = L
    dimvals = all_vals[dim]
    
    _, mask, binned, binned_L = common.binned_max(dimvals, lvals, start, stop, bin_amt) 
    if binned_L[mask].shape[0] == 0:
        return []
    smoothed = common.smooth(binned_L[mask])
    
    #Check to see if there is even a possible peak
    median = np.median(smoothed)
    peak = np.max(smoothed)
    print peak, median
    if peak < 0.999 * median:
        #some criteria
        return []
    
    index = np.where(smoothed == peak)[0]
    print index
    print "peak is at"
    print all_vals[dim, index]
    
    mins = common.compute_mins(binned[mask], smoothed)
    maxes = common.compute_maxes(binned[mask], smoothed)
    intervals = common.compute_intervals(mins, maxes)
    intervals = np.floor(intervals).astype("int")
    
    if intervals.shape[0] == 0:
        return []
    
    #a few base cases
    
    if debug_plots:
        plt.figure(figsize=(20,20))
        f, ax = plt.subplots(3,3)
        f.tight_layout()
        counter = 0
        for sublst in ax:
            for subplot in sublst:
                if counter >= intervals.shape[0]:
                    continue
                begin,end = intervals[counter]
                counter += 1
                mask = np.where(( x > begin) & (x < end))[0]
                mycol = uniqueish_color()
                subplot.plot(all_vals[next_dim(dim), mask], L[mask], ',',color=mycol)
                subplot.set_title("%f - %f"%(begin,end))
        plt.title("DIM = %d"%dim)
    new_bounds = []
    
    print intervals
    
    for nstart, nstop in intervals:
        b = np.zeros((2,2))
        b[dim] = [start, stop]
        b[next_dim(dim)] = nstart, nstop
        new_bounds.append(b)
    print "new bounds"
    print new_bounds
    return [(dim, [start, stop], get_peaks(b, next_dim(dim), debug_plots=debug_plots)) for b in new_bounds]

In [7]:
initial_bounds = np.array([[0, width], [0, height]])
k = get_peaks(initial_bounds, 0, debug_plots=False)

-105713.910729 -108217.655568
[220]
peak is at
[ 99.027232]
[[  2  27]
 [ 27  43]
 [ 43  98]
 [ 98 135]
 [135 169]]
new bounds
[array([[   0.,  200.],
       [   2.,   27.]]), array([[   0.,  200.],
       [  27.,   43.]]), array([[   0.,  200.],
       [  43.,   98.]]), array([[   0.,  200.],
       [  98.,  135.]]), array([[   0.,  200.],
       [ 135.,  169.]])]
-107205.351043 -108572.02762
[328]
peak is at
[ 101.322625]
[[ 2  4]
 [ 4  5]
 [ 5  8]
 [ 8 10]
 [10 11]
 [11 12]
 [12 13]
 [13 14]
 [14 16]
 [16 18]
 [18 20]
 [20 23]
 [23 25]]
new bounds
[array([[  2.,   4.],
       [  2.,  27.]]), array([[  4.,   5.],
       [  2.,  27.]]), array([[  5.,   8.],
       [  2.,  27.]]), array([[  8.,  10.],
       [  2.,  27.]]), array([[ 10.,  11.],
       [  2.,  27.]]), array([[ 11.,  12.],
       [  2.,  27.]]), array([[ 12.,  13.],
       [  2.,  27.]]), array([[ 13.,  14.],
       [  2.,  27.]]), array([[ 14.,  16.],
       [  2.,  27.]]), array([[ 16.,  18.],
       [  2.,  27.]]), ar

In [8]:
k

[(0,
  [0, 200],
  [(1, [2.0, 27.0], [(0, [2.0, 4.0], []), (0, [2.0, 4.0], [])]),
   (1, [2.0, 27.0], []),
   (1,
    [2.0, 27.0],
    [(0, [5.0, 8.0], []), (0, [5.0, 8.0], []), (0, [5.0, 8.0], [])]),
   (1, [2.0, 27.0], [(0, [8.0, 10.0], []), (0, [8.0, 10.0], [])]),
   (1, [2.0, 27.0], []),
   (1, [2.0, 27.0], []),
   (1, [2.0, 27.0], []),
   (1, [2.0, 27.0], []),
   (1, [2.0, 27.0], [(0, [14.0, 16.0], []), (0, [14.0, 16.0], [])]),
   (1, [2.0, 27.0], [(0, [16.0, 18.0], []), (0, [16.0, 18.0], [])]),
   (1, [2.0, 27.0], [(0, [18.0, 20.0], []), (0, [18.0, 20.0], [])]),
   (1,
    [2.0, 27.0],
    [(0, [20.0, 23.0], []), (0, [20.0, 23.0], []), (0, [20.0, 23.0], [])]),
   (1, [2.0, 27.0], [(0, [23.0, 25.0], []), (0, [23.0, 25.0], [])])]),
 (0,
  [0, 200],
  [(1, [27.0, 43.0], []),
   (1, [27.0, 43.0], []),
   (1, [27.0, 43.0], []),
   (1, [27.0, 43.0], [(0, [30.0, 32.0], []), (0, [30.0, 32.0], [])]),
   (1, [27.0, 43.0], [(0, [32.0, 34.0], []), (0, [32.0, 34.0], [])]),
   (1, [27.0, 43.0]

Below is still a work in progress

In [9]:
test_tree = [(0, [72.0, 94.0], [(1, [85.0, 86.0], [])])]

In [10]:
test_tree2 = [(0,
  [0, 200],
  [(1, [6.0, 16.0], []),
   (1, [6.0, 16.0], []),
   (1, [6.0, 16.0], []),
   (1, [6.0, 16.0], []),
   (1, [6.0, 16.0], []),
   (1, [6.0, 16.0], []),
   (1, [6.0, 16.0], []),
   (1, [6.0, 16.0], []),
   (1, [6.0, 16.0], []),
   (1, [6.0, 16.0], [])])]

In [11]:
pdb on

Automatic pdb calling has been turned ON


In [12]:
def post_process(tree, depth=0):
    curr_dim = tree[0]
    dim_bounds = tree[1]
    sub_bounds = tree[2]
    if sub_bounds == []:
        return []
    my_sub_bounds = []
    for sb in sub_bounds:
        if sb[-1] == []:
            my_sub_bounds.append(sb)
    return_vals = [(depth, [dim_bounds, sb[1]]) for sb in my_sub_bounds]
    for sub_tree in sub_bounds:
        return_vals += post_process(sub_tree, depth+1)
    return return_vals

def get_sources(tree):
    res = []
    for subtree in k:
        res += post_process(subtree)
    return res

In [22]:
def get_sources(tree):
    res = [post_process(subtree) for subtree in k]
    return_val = []
    for part in res:
        entry = max(part, key = lambda item: item[0])
        deepest = entry[0]
        
        #now that we know the deepest, we can go and keep only the last calls
        keep = filter(lambda item: item[0] == deepest, part)
        return_val += keep
    return return_val
        

In [23]:
get_sources(k)

[(1, [[2.0, 27.0], [2.0, 4.0]]),
 (1, [[2.0, 27.0], [2.0, 4.0]]),
 (1, [[2.0, 27.0], [5.0, 8.0]]),
 (1, [[2.0, 27.0], [5.0, 8.0]]),
 (1, [[2.0, 27.0], [5.0, 8.0]]),
 (1, [[2.0, 27.0], [8.0, 10.0]]),
 (1, [[2.0, 27.0], [8.0, 10.0]]),
 (1, [[2.0, 27.0], [14.0, 16.0]]),
 (1, [[2.0, 27.0], [14.0, 16.0]]),
 (1, [[2.0, 27.0], [16.0, 18.0]]),
 (1, [[2.0, 27.0], [16.0, 18.0]]),
 (1, [[2.0, 27.0], [18.0, 20.0]]),
 (1, [[2.0, 27.0], [18.0, 20.0]]),
 (1, [[2.0, 27.0], [20.0, 23.0]]),
 (1, [[2.0, 27.0], [20.0, 23.0]]),
 (1, [[2.0, 27.0], [20.0, 23.0]]),
 (1, [[2.0, 27.0], [23.0, 25.0]]),
 (1, [[2.0, 27.0], [23.0, 25.0]]),
 (1, [[27.0, 43.0], [30.0, 32.0]]),
 (1, [[27.0, 43.0], [30.0, 32.0]]),
 (1, [[27.0, 43.0], [32.0, 34.0]]),
 (1, [[27.0, 43.0], [32.0, 34.0]]),
 (1, [[27.0, 43.0], [36.0, 38.0]]),
 (1, [[27.0, 43.0], [36.0, 38.0]]),
 (1, [[27.0, 43.0], [38.0, 40.0]]),
 (1, [[27.0, 43.0], [38.0, 40.0]]),
 (2, [[58.0, 96.0], [61.0, 63.0]]),
 (2, [[58.0, 96.0], [63.0, 65.0]]),
 (2, [[58.0, 96.0], [6